In [ ]:
'''Join acc and gyro data then export to csv'''
def join_export(acc_frame, gyro_frame, export_name):
    # Extract only the gyroscope data from the gyro dataframe
    # This assumes the time components of each are aligned
    join_frame = acc_frame.join(gyro_frame[['GYROSCOPE X (rad/s)', 'GYROSCOPE Y (rad/s)', 'GYROSCOPE Z (rad/s)']])
    path = 'imu_data/' + export_name
    join_frame.to_csv(path, index=False)

In [ ]:
# Run on everything

terrains = ['Concrete', 'Carpet', 'Linoleum']
locations = ['left', 'right', 'Frame9250']

for terrain in terrains:
    for location in locations:
        if (location == 'Frame9250'):
            filename = 'FrameMiddle' + terrain + '.csv'
        else:
            filename = 'Wheel' + location.capitalize() + terrain + '.csv'
        gyro = raw_datasets[location + 'Gyro' + terrain]
        acc = raw_datasets[location + 'Acc' + terrain]
        join_export(acc, gyro, filename)

In [ ]:
# Attempting to calculate the angular velocity of the wheelchair using the rotational velocity of each wheel

wheel_radius = 0.60 #m
wheel_sep = 0.502 #m

raw_datasets['CalculatedLinoleum'] = pd.DataFrame()

raw_datasets['calculatedLinoleum']['YYYY-MO-DD HH-MI-SS_SSS'] = raw_datasets['leftGyroLinoleum']['YYYY-MO-DD HH-MI-SS_SSS']
raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'] = raw_datasets['leftGyroLinoleum']['GYROSCOPE Z (rad/s)'] - raw_datasets['rightGyroLinoleum']['GYROSCOPE Z (rad/s)'] 
raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'] = raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'].apply(lambda x: x*(wheel_radius/wheel_sep))

label_compare('calculatedLinoleum', 'Frame9250GyroLinoleum', 'GYROSCOPE Z (rad/s)')

In [ ]:
# Convert time received to time since start
for label in ('Concrete', 'Carpet', 'Linoleum'):
    for mount in ('WheelLeft', 'WheelRight'):
        start = Decimal(raw_datasets[mount+label]['Time received in s'][0])
        raw_datasets[mount+label]['Time received in s'] = raw_datasets[mount+label]['Time received in s'].apply(lambda x: float((Decimal(x) - start) * 1000))
        raw_datasets[mount+label].rename(columns = {'Time received in s': 'Time since start in ms '}, inplace=True)

In [ ]:
# Standardize order of columns
for label in raw_datasets.keys():
    raw_datasets[label] = raw_datasets[label][get_columns(label)]

In [ ]:
# Export everything to CSV
for label, dataset in raw_datasets.items():
    dataset.to_csv('imu_data/'+label+'.csv', index=False)

In [ ]:
# Convert string format to epoch time (newer data only)
    if any(s in dataset_label for s in ('Phone', 'Asphalt', 'Grass', 'Gravel', 'Sidewalk')):
        dataset['Epoch Time'] = dataset['Epoch Time'].apply(datetime.strptime, 
                                                            args=("%Y-%m-%d %H:%M:%S:%f",))
        dataset['Epoch Time'] = dataset['Epoch Time'].apply(datetime.timestamp)

In [ ]:
# Need to convert everything to a single pandas dataset
def to_single_dataframe(datasets, feat_names):
    # Create the columns of the array
    
    columns = ['Label']
    for j in range(N_DATA_COL):
        for name in feat_names:
            columns.append(dataset_columns[j] + ' ' + name)
    dataframe = pd.DataFrame(columns=columns, index=[])
    
    for i, dataset in enumerate(datasets):
        set_dataframe = pd.DataFrame(columns)
        
        for j, direction in enumerate(dataset[1:]):
            # Direction labelled columns
            direction_columns = ['Label']
            for name in feat_names:
                direction_columns.append(dataset_columns[j] + ' ' + name)
            
            # Data directions into a single dataframe
            direction.columns = direction_columns
            set_dataframe = set_dataframe.append(direction)
            
        # Combine datasets into a single dataframe
        dataframe.append(dataset)
    
    return dataframe

In [ ]:
'Add labels to a dataset'
def insert_labels(datasets, windowed=False):
    # Add to each dataframe of a dataset
    for i, (label, dataset) in enumerate(datasets.items()):
        # Either formatted in windows (transforms) or dict->dict (features)
        if (windowed):
            for window_df in dataset:
                labels = [get_terrain(label) for _ in range(len(window_df))]
                window_df.insert(0, 'Label', labels)
                window_df.set_index('Label')
        
        else:
            for dirn_label, dirn_df  in dataset.items():
                labels = [get_terrain(label) for _ in range(len(dirn_df))]
                dirn_df.insert(0, 'Label', labels)
                dirn_df.set_index('Label')